In [ ]:
import scanpy as sc
import numpy as np
from vitessce.data_utils import rgb_img_to_ome_zarr, VAR_CHUNK_SIZE
from vitessce import (
    VitessceConfig,
    AnnDataWrapper,
    ImageOmeZarrWrapper,
)
from os.path import join

In [ ]:
import marimo as mo

In [ ]:
output_img = join("data", "V1_Human_Lymph_Node.ome.zarr")
output_adata = join("data", "V1_Human_Lymph_Node.anndata.zarr")

# Vitessce Widget Tutorial

Note: This approach to storing images within AnnData objects is no longer recommended now that [SpatialData](https://spatialdata.scverse.org/en/stable/) has been introduced.

# Visualization of AnnData object containing an image in `uns`

In [ ]:
adata = sc.datasets.visium_sge(sample_id="V1_Human_Lymph_Node", include_hires_tiff=True)

# Write img_arr to OME-Zarr.
# Need to convert images from interleaved to non-interleaved (color axis should be first).
img_hires = adata.uns['spatial']['V1_Human_Lymph_Node']['images']['hires']
img_arr = np.transpose(img_hires, (2, 0, 1))
# Convert values from [0, 1] to [0, 255].
img_arr *= 255.0

# First, save the image to an OME-Zarr image format
rgb_img_to_ome_zarr(img_arr, output_img, axes="cyx", chunks=(1, 256, 256), img_name="Image")
# Second, save the AnnData object to Zarr format
adata.write_zarr(output_adata, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

In [ ]:
vc = VitessceConfig(schema_version="1.0.17", name="AnnData with image")
dataset = vc.add_dataset("My dataset").add_object(
    AnnDataWrapper(
        adata_path=output_adata,

    )
).add_object(
    ImageOmeZarrWrapper(
        img_path=output_img,
    )
)

spatial_view = vc.add_view("spatialBeta", dataset=dataset)
lc_view = vc.add_view("layerControllerBeta", dataset=dataset)

vc.layout(spatial_view | lc_view);

In [ ]:
vw = vc.widget()
vw